<a href="https://colab.research.google.com/github/rrrohit1/fine-tuning-llama/blob/main/fine_tuning_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install Packages

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

**Bitsandbytes**: Used for quantization. Weights are in float type. So, we quantize it to int8 from float16/32 as we have limited GPU.

# Step 2: Import packages

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import PeftModel, PeftConfig
from trl import SFTTrainer

## Using LLama 2
The following prompt template is used to chat with the model.

- System Prompt: guides the model (optional)
- User prompt: give instructions (required)
- Model answer: (required)



```
<s>[INST] <<SYS>>
System prompt
<</SYS>>

User prompt [/INST] Model Answer </s>
```

## Reformat the instruction dataset to follow Llama 2 template

Llama 2 has 7 billion weights.
Fine-tuning is not possible due to limited GPU, hence we use PEFT techniques like LoRA.

# Step 3: Fine-tune process

1. Load a llama2-7b-chat-hf model
2. Train it on the dataset which will produce our fine-tuned model Llama-2-7b-chat-finetune

QLoRA: used rank 64 with a scaling parameter of 16. The Llama 2 model is directly loaded in 4-bit precision using NF4 type and train it for one epoch.

In [ ]:

model_name = 'NousResearch/Llama-2-7b-chat-hf'

dataset_name = 'mlabonee/guanaco-llama2-1k'

new_model = 'Llama-2-7b-chat-finetune'


# LoRA prarameters

lora_r = 6

lora_alpha = 16

lora_dropout = 0.1


# Quantization?

use_4bit = True

bnb_4bit_compute_dtype = "float16"

bnb_4bit_quant_type = "nf4"

use_nested_quant = True

